In [1]:
import gc
import csv
from scipy.misc import imread
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import sklearn
import json
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import model_from_json
from keras.models import Sequential
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Dense, Dropout, Activation, Flatten, ELU, Lambda
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array


Using TensorFlow backend.


In [ ]:
def my_generator(data,batch_size=1):
   while True:
       print("new iteration")
       shuffle = np.random.choice(range(data.shape[0]),data.shape[0],replace=False)
       data = data[shuffle]
       for i in range(0,X.shape[0],batch_size):
           yield (data[i:i+batch_size])

In [ ]:
X = np.array([1,2,3,4,5,6,7,8,9,10])
y = np.array(['a','b','c','d','e','f','g','h','i','j'])

g = my_generator(y,batch_size=2)
for i in range(5):
   a = next(g)
   print(a)

In [ ]:
def data_generator1(X,y,batch_size=1):
   while True:
       print("starting with batch_size ",batch_size)
       print()
       Xs, ys = shuffle_data(X,y)
       for i in range(0,X.shape[0],batch_size):
           #print(ys[i:i+batch_size])
           yield (Xs[i:i+batch_size], ys[i:i+batch_size])

In [11]:
def data_generator(X,y,batch_size=1):
   print("in data_generator")
   total = len(X)
  
   while True:
       print("starting with batch_size ",batch_size)
       print()
       Xs, ys = shuffle_data(X,y)
       for i in range(0,total,batch_size):
           print(ys[i:i+batch_size])
           yield (Xs[i:i+batch_size], ys[i:i+batch_size])
            
            

In [ ]:
X = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
y = np.array([101,102,103,104,105,106,107,108,109,110,111,112])

size = len(X)
batch_size = 2
num_iters = round(size/batch_size)

g = data_generator1(X,y,batch_size)
for i in range(num_iters):
   a,b = next(g)
   print(a)
   print(b)
   print('=====') 

In [12]:
model = get_model()

total = len(X)
batch_size = 4

    
history = model.fit_generator(data_generator(X,y,batch_size), samples_per_epoch = total, nb_epoch = 3,
verbose=2, callbacks=[], validation_data=None, class_weight=None, nb_worker=1)

print(model.predict(X_val[0:4]))

print("Original values...")
for i in range(3):
    print(y_val[i])

in data_generatorEpoch 1/3

starting with batch_size  4

[[  4.]
 [ 16.]
 [  8.]
 [  1.]]
[[ 20.]
 [ 18.]
 [  3.]
 [  5.]]
[[  2.]
 [ 15.]
 [ 10.]
 [ 13.]]
[[ 11.]
 [ 19.]
 [ 17.]
 [  7.]]
[[  9.]
 [  6.]
 [ 14.]
 [ 12.]]
starting with batch_size  4

[[  8.]
 [  1.]
 [ 11.]
 [ 15.]]
[[ 13.]
 [ 20.]
 [ 14.]
 [  7.]]
[[ 19.]
 [  6.]
 [ 17.]
 [  5.]]
[[ 10.]
 [ 12.]
 [  3.]
 [ 18.]]
[[  2.]
 [  4.]
 [ 16.]
 [  9.]]
starting with batch_size  4

[[ 20.]
 [ 16.]
 [ 10.]
 [ 13.]]
[[  9.]
 [  5.]
 [ 15.]
 [  7.]]
[[ 14.]
 [  3.]
 [ 12.]
 [ 17.]]
[[ 18.]
 [  4.]
 [  2.]
 [ 19.]]
[[ 11.]
 [  1.]
 [  6.]
 [  8.]]
2s - loss: 72.9159 - acc: 0.1000
Epoch 2/3
starting with batch_size  4

[[ 11.]
 [  5.]
 [  7.]
 [ 17.]]
[[ 12.]
 [  4.]
 [  9.]
 [ 10.]]
[[ 18.]
 [ 19.]
 [  6.]
 [ 14.]]
[[ 16.]
 [  3.]
 [  2.]
 [  1.]]
[[ 20.]
 [  8.]
 [ 15.]
 [ 13.]]
2s - loss: 46.8865 - acc: 0.0500
Epoch 3/3
starting with batch_size  4

[[ 15.]
 [  4.]
 [ 10.]
 [  7.]]
[[ 14.]
 [ 17.]
 [ 11.]
 [  3.]]
[[ 18.]
 [ 19.]

In [6]:
path = "data/"
filename = "dummy.csv"   
model_name = "dumm"

In [4]:
def load_csv():
    image_paths = {}
    steering_angles = {}
    
    with open(path+filename, 'r') as f:
        reader = csv.reader(f)
        next(reader, None) 

        index = 0
        for row in reader:
           image_paths[index] = row[0]
           steering_angles[index] = float(row[3]) 
           index+=1
    return image_paths, steering_angles

def load_data():
    image_paths, steering_angles = load_csv()
    size = len(image_paths)
    shape = imread(path+image_paths[0]).shape
    imshape = shape
    
    X = np.zeros(shape=(size,shape[0],shape[1],shape[2]))
    y = np.zeros(shape=(size,1)).astype(float)
    
    for i in range(size):
        img = imread(path+image_paths[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = img[55:140:,:,:]
        #img = cv2.resize(img,(320,160))
        X[i] = img_to_array(img)
        y[i] = steering_angles[i]
        i+=1
    
    return X, y


def shuffle_data(X, y):
    size = len(X)
    X_shuf = np.empty(shape=X.shape)
    y_shuf = np.empty(shape=y.shape)
    
    index_shuf = np.arange(len(X))
    
    np.random.shuffle(index_shuf)
    
    index = 0
    
    for i in index_shuf:
        X_shuf[index] = X[i]
        y_shuf[index] = y[i]
        index+=1
        
    return X_shuf, y_shuf 


def get_model():
    model = Sequential()

    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=X_train[0].shape,output_shape=X_train[0].shape))

    model.add(Convolution2D(24, 5, 5, border_mode='same', init="normal"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(ELU())

    model.add(Convolution2D(36, 5, 5, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(ELU())

    model.add(Convolution2D(48, 5, 5, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(ELU())

    model.add(Convolution2D(64, 5, 5, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(ELU())


    model.add(Flatten())
    model.add(Dense(100))
    model.add(ELU())

    model.add(Dense(50))
    model.add(ELU())

    model.add(Dense(10))
    model.add(ELU())

    model.add(Dense(1))

    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error',optimizer=opt,metrics=['accuracy'])
    
    return model

In [8]:
X,y = load_data()
Xs, ys = shuffle_data(X,y)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(Xs, ys, test_size=0.25, random_state=42)


In [ ]:
model = get_model()
history = model.fit(X_train, y_train, nb_epoch=3, batch_size=20, verbose = 1)
print(model.predict(X_val[0:4]))

print("Original values...")
for i in range(3):
    print(y_val[i])

In [ ]:

model.save_weights(model_name+".h5")
model_json = model.to_json()
with open(model_name+".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5

print("Saved model to disk")



In [ ]:
checkpoint = ModelCheckpoint("model-{epoch:02d}.h5", monitor='loss',
    verbose=1, save_best_only=False, mode='max')

    callbacks_list = [checkpoint]
    
history = model.fit_generator(batchgen(x_train_less, y_train_less),         
        samples_per_epoch = samples_per_epoch, nb_epoch = nb_epoch,
                    verbose=1, max_q_size = max_q_size, callbacks=callbacks_list,         
        validation_data=None, class_weight=None,
                     pickle_safe=False)



In [ ]:
for idx in range(5):
  ... 
  model = get_model()
  history = model.fit_generator(train_data_generator, samples_per_epoch=20480,
                  nb_epoch=6, validation_data=valid_data_generator,
                  nb_val_samples=val_size)
                  
  ... 
  val_loss = history.history['val_loss'][0]
  # Save your model 

 # Adaptive throttle - Both Track
    if (abs(float(speed)) < 10):
        throttle = 0.5 
    else:
        # When speed is below 20 then increase throttle by speed_factor
        if (abs(float(speed)) < 25):
            speed_factor = 1.35
        else:
            speed_factor = 1.0 
        if (abs(steering_angle) < 0.1): 
            throttle = 0.3 * speed_factor
        elif (abs(steering_angle) < 0.5):
            throttle = 0.2 * speed_factor
        else:
            throttle = 0.15 * speed_factor
    print(steering_angle, throttle, speed)